In [ ]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split, col
from pyspark.sql.functions import last,max

In [ ]:
spark = SparkSession.builder.appName("phase1").getOrCreate()

In [ ]:
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.123").option("port",9000).load()

In [ ]:
point = line.select(explode(split(line.value," ")).alias("id"))
point = point.withColumn('id', split(point["id"], '\,')[2].cast(types.DoubleType()))
point = point.withColumn('lat', split(point["id"], '\,')[1].cast(types.DoubleType()))
point = point.withColumn('lon', split(point["id"], '\,')[0].cast(types.DoubleType()))
point = point.groupBy('lat','lon').count()

def batchProcess1(sdf, epoch_id):
    df = sdf.toPandas()
    print(df.head())
    if df.empty:
        pass
    else:
        df.to_csv('tmp/s1/'+str(epoch_id)+'.csv',header=False,index=False)
        
point.writeStream.outputMode("complete").foreachBatch(batchProcess1).start() 

In [ ]:
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('lat', split(ppoint["id"], '\,')[3].cast(types.FloatType()))
ppoint = ppoint.withColumn('lon', split(ppoint["id"], '\,')[2].cast(types.FloatType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
ppoint = ppoint.groupby('id').agg(max('timestamp'), last('lat'), last('lon')).toDF('id','time','lat','lon').drop('id','time')

def batchProcess2(sdf, epoch_id):
    df = sdf.toPandas()
    if df.empty:
        pass
    else:
        minloc = float(df.iloc[0]['lon'] - 0.25)
        maxloc = float(df.iloc[0]['lon'] + 0.25)
        minlat = float(df.iloc[0]['lat'] - 0.25)
        maxlat = float(df.iloc[0]['lat'] + 0.25)
        f = open('tmp/s2/'+str(epoch_id)+'.csv','w')
        f.write(str(minloc)+','+str(maxloc)+','+str(minlat)+','+str(maxlat)+'\n')
        f.close()

ppoint.writeStream.outputMode("complete").foreachBatch(batchProcess2).start()   